# Group API Project
Crime vs. Unemployment Rates

What if any relationship is there between national property crimes rates versus unemployment rates in 2010? in 2020?

Is there a noticeable difference between the years? Is there a potential correlation between crime and unemployment rates? Is there a stronger correlation in one subgroup of property crimes than the others?

In [5]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from pprint import pprint
import citipy
import json
from citipy import citipy
import seaborn as sns
import prettytable

#Federal Bureau of Investigation
#Crime Data Explorer
FBI_api_key="HXzuNSaaIO46x38N0iVVKpMDgrP6GG3mxBcacgRH"

#U.S. Bureau of Labor Statistics
BLS_api_key="489c914f30b34dac8ebc65feb25a15df"

## Performing the API Calls and Collecting the Data

In [6]:
#FBI JSON request
#base_url: https://api.usa.gov/crime/fbi/sapi/api/nibrs/{burglary}/offense/national/{count}

# I think we want to use base_url = https://api.usa.gov/crime/fbi/sapi/api/data/nibrs/larceny-theft-offenses/offender/national/count
# This url will give us the number of offenders with larceny offenses ^^^^^

fbi_burglary = requests.get('https://api.usa.gov/crime/fbi/sapi/api/nibrs/burglary/offense/national/count/?limit=1&api_key=HXzuNSaaIO46x38N0iVVKpMDgrP6GG3mxBcacgRH', data=data, headers=headers)
fbi_json_data_bur = json.loads(fbi_burglary.text)
print(fbi_json_data_bur)

# I think we are going to need population data as well so that we can create a percent of larceny offenders to total
#population.  That way we can effectively compare it to the unemployment percentage.

NameError: name 'data' is not defined

In [ ]:
#create empty dictionary to store data from API call
burglary_data = {'Year':[],
                'Number':[],
                'Category':[]}
#create years list for looping]
years=[2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

for year in years:
    
    base_url = "https://api.usa.gov/crime/fbi/sapi/api/nibrs/{burglary}/offense/national/{count}" + FBI_api_key

    try:
        burglary_data['Year'].append(fbi_json_data_bur['data'][0]['data_year'])
        burglary_data['Number'].append(fbi_json_data_bur['data'][0]['value'])
        burglary_data['Category'].append(fbi_json_data_bur['data'][0]['key'])
    
        print('burglary data processing for ' + year)
        
    except:
        print(year + ' not found')
        pass

In [25]:
raw_fbi_data_bur = []
raw_fbi_data_bur.append(fbi_json_data_bur)
fbi_data_unformed_bur = pd.json_normalize(raw_fbi_data_bur, sep='_')
#fbi_data_unformed_bur = fbi_data_unformed_bur.set_index("ui_type")
fbi_data_unformed_bur.to_csv('fbi_data_unformed_bur_csv.csv')

In [7]:
#BLA JSON request
#base_url="https://api.bls.gov/publicAPI/v2/timeseries/data/?registrationkey=489c914f30b34dac8ebc65feb25a15df&catalog=true&startyear=2010&endyear=2020&calculations=true&annualaverage=true&aspects=true"

# changed start year to 2011 so we can get 2020 data in the API call
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['CUUR0000SA0','SUUR0000SA0'],"startyear":"2011", "endyear":"2020"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.txt','w')
    output.write (x.get_string())
    output.close()

In [9]:
print(json_data)

{'status': 'REQUEST_SUCCEEDED', 'responseTime': 271, 'message': ['Year range has been reduced to the system-allowed limit of 10 years.'], 'Results': {'series': [{'seriesID': 'CUUR0000SA0', 'data': [{'year': '2019', 'period': 'M12', 'periodName': 'December', 'value': '256.974', 'footnotes': [{}]}, {'year': '2019', 'period': 'M11', 'periodName': 'November', 'value': '257.208', 'footnotes': [{}]}, {'year': '2019', 'period': 'M10', 'periodName': 'October', 'value': '257.346', 'footnotes': [{}]}, {'year': '2019', 'period': 'M09', 'periodName': 'September', 'value': '256.759', 'footnotes': [{}]}, {'year': '2019', 'period': 'M08', 'periodName': 'August', 'value': '256.558', 'footnotes': [{}]}, {'year': '2019', 'period': 'M07', 'periodName': 'July', 'value': '256.571', 'footnotes': [{}]}, {'year': '2019', 'period': 'M06', 'periodName': 'June', 'value': '256.143', 'footnotes': [{}]}, {'year': '2019', 'period': 'M05', 'periodName': 'May', 'value': '256.092', 'footnotes': [{}]}, {'year': '2019', 

In [ ]:
#create empty dictionary to store data from API call
unemp_data = {'Year':[],
              'Month':[],
              'Value':[]}

# This won't work because I need a 'year' as part of the url to do this.  How can we iterate through the data with a 
# 'try'/'except'? vvvvv

for year in years:
    
    base_url = "https://api.bls.gov/publicAPI/v2/timeseries/data/?registrationkey=489c914f30b34dac8ebc65feb25a15df&catalog=true&startyear=2010&endyear=2020&calculations=true&annualaverage=true&aspects=true"
" + BLS_api_key

    try:
        unemp_data['Year'].append(json_data['data'][0]['data_year'])
        unemp_data['Month'].append(json_data['data'][0]['value'])
        unemp_data['Value'].append(json_data['data'][0]['key'])
        
        print('unemployment data processing for ' + year)
    
    except:
        print(year + ' not found')
        pass


#code from Week 5 to help me remember the steps
#cleaning the data
cities_weather_unformed = pd.json_normalize(raw_weather, sep='_')
cities_weather_unformed = cities_weather_unformed.drop(cities_weather_unformed[cities_weather_unformed.message == "city not found"].index)

cities_weather_unformed = cities_weather_unformed.set_index("name")
cities_weather_unformed.to_csv('cities_weather_unformed_csv.csv')

#building a focused dataframe
cities_weather = cities_weather_unformed.filter(["sys_country", "coord_lon", "coord_lat", "main_pressure", "main_temp_max", "wind_speed", "main_humidity", "clouds_all"], axis=1)
cities_weather = pd.DataFrame(cities_weather)

#export to a csv for confirmation
cities_weather.to_csv('cities_weather_csv.csv')

#print a sample to show it works
cities_weather.head(75)

#code from Week 5 to help me remember the steps

raw_weather = []

base_url="http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key

try:
    for city in cities:
        req = requests.get(base_url + f'&q={city}')
        json_data = req.json()
        raw_weather.append(json_data)
        print("Processing "+ city)

except requests.exceptions.RequestException as e:
        print("Processing "+ city + "failed.")
